<a href="https://colab.research.google.com/github/winkler-eva/CovidClassifierBIOINF/blob/main/CovidClassifierVoila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai --upgrade -q

     |████████████████████████████████| 194kB 8.1MB/s 
     |████████████████████████████████| 12.8MB 280kB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [3]:
from fastai.vision.all import *
import ipywidgets as widgets
import requests

In [4]:
# https://drive.google.com/file/d/1aF6xFbEFpequCtiqTHjlIblPg1NISix_/view?usp=sharing
def download_file_from_google_drive(id, destination):
  URL = "https://docs.google.com/uc?export=download"
  session = requests.Session()
  response = session.get(URL, params = { 'id' : id }, stream = True)
  token = get_confirm_token(response)
  if token:
      params = { 'id' : id, 'confirm' : token }
      response = session.get(URL, params = params, stream = True)
  save_response_content(response, destination)    

def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value
  return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [5]:
file_id = '1aF6xFbEFpequCtiqTHjlIblPg1NISix_'
destination = './covid_classifier.pkl'
download_file_from_google_drive(file_id, destination)

<h1>Covid Klassifizier</h1>
<p>Folgende CT-Bilder können erkannt werden:</p>
<ul>
  <li>Gesunde Lunge (Non-Covid)</li>
  <li>Kranke Lunge (Covid)</li>
</ul>

In [7]:
path = Path()
inference = load_learner(path/'covid_classifier.pkl', cpu=True)

In [8]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Klassifizieren')
out_pl = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  pred, pred_idx, probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label("Select lung CT:"),
             btn_upload, btn_run, out_pl, lbl_pred])